# Prepare Initial Datasets

In [1]:
from functions_SFMA import *

## Load W matrices

In [2]:
num_N = 6 # number of rows of a matrix 

In [3]:
n_bit = num_N*num_K # number of spins (qubits)
val_dim = f'd{n_bit}'

# Load W matrices and mean and std values
folder0 =  '20241227_pFMA_Wmatrix/results/z900/' 
name0   =  f'C6_{num_N}x2'  
init_file = f'{folder0}/{name0}.pickle.bz2'
with bz2.open(init_file, 'rb') as f:
        param = pickle.load(f)

# Load the W matrices 
list_Wmatrix = [] # list for collecting the W matrices 
for w in param['tdata']:
     list_Wmatrix.append(w) # append the W matrix to list_Wmatrix

# Load the mean and std values
obj_mean = param['obj_func_mean'] # array of the mean values of the objective functions (output values)
obj_std = param['obj_func_std']   # array of the standard deviation (std) values of the outputs

## Create and save initial datasets

In [4]:
#################### Parameters for initial datasets and FM models
n_in = n_bit # size of initial datasets 
k_FM = int(n_bit/2)-1 # hyperparameter for FM models 
val_k = str(k_FM)
################# # Create initial datasets by SA and standardization.
val_anneal = 'SA'
val_nonstandard = 'nonstandard'
val_standard = 'standard'
folder_data =  f'final_data_file_{n_bit}'  # folder for saving the datasets. 
for num_W in range(d_W): # create an initial dataset for each W matrix
    val_num_W = f'W_{num_W}'
    Matrix_W = list_Wmatrix[num_W] # W matrix
    mean = obj_mean[num_W] # mean value
    std = obj_std[num_W] # standard deviation (std) value
    #
    minBB_1st = param['optimal']['opt'][num_W][0] # 1st optimal objective 
    minBB_2nd = param['optimal']['opt'][num_W][1] # 2nd optimal objective 
    minBB_mid2 = param['optimal']['mid2'][num_W] # objective value of the best candidate solution obtained by the original method: mid2 optimal value
    ### create initial datasets
    x_in, y_in,  = init_training_data(Matrix_W, num_N, num_K, n_bit, n_in, nin_seed) # generating an inital dataset
    y_in_nonstandard = y_in  # non-standardized output value
    y_in_standard = standard(y_in, mean, std) # standardized output value
    ################# Train FM models.
    #
    torch.manual_seed(seed_intial_FMTorch) # set a random seed for reproducibility 
    model_nonstandard = TorchFM(d=n_bit, k=k_FM, Stand_Dev=1) # instantiate a FM model 
    trained_FM_nonstandard_set = train_FM_model(x_in, y_in_nonstandard, model_nonstandard) # train a FM model
    #
    torch.manual_seed(seed_intial_FMTorch) # set a random seed for reproducibility 
    model_standard = TorchFM(d=n_bit, k=k_FM, Stand_Dev=std) # instantiate a FM model 
    trained_FM_standard_set = train_FM_model(x_in, y_in_standard, model_standard) # train a FM model
    #
    ####### SA
    _, _, xstar_nonstandard, ystar_nonstandard = simulated_anneal(trained_FM_nonstandard_set,  Matrix_W, num_N, num_K)
    # xstar_nonstandard: best candiate solution
    # ystar_nonstandard: output value of the best candiate solution: the minimum value
    
    _, _, xstar_standard, ystar_standard = simulated_anneal(trained_FM_standard_set,  Matrix_W, num_N, num_K)
    # xstar_standard: best candiate solution
    # ystar_standard: output value of the best candiate solution: the minimum value
    
    ####### Augument the dataset with the solution. 
    xs_int_nonstandard = np.vstack((x_in, xstar_nonstandard)) # augmented x
    ys_int_nonstandard = np.append(y_in, ystar_nonstandard) # augmented y
    xs_int_standard = np.vstack((x_in, xstar_standard)) # augmented x
    ys_int_standard = np.append(y_in, ystar_standard) # augmented y

    
    ## Save the datasets without standardization.
    dictionary_nonstandard = { 'xs_int': xs_int_nonstandard, 'ys_int': ys_int_nonstandard,
              'Matrix_W': Matrix_W, 'num_N': num_N, 
              'mean': mean, 'std': std, 'minBB_1st': minBB_1st, 'minBB_2nd': minBB_2nd, 'minBB_mid2': minBB_mid2}
                               

    file_nonstandard = f'{folder_data}/initial_Data_{val_num_W}_{val_dim}_{val_nonstandard}_{val_k}_{val_anneal}.bz2'
                                                                           
    with open(file_nonstandard, 'wb') as fl_nonstandard:
        pickle.dump(dictionary_nonstandard, fl_nonstandard)

    ## Save the datasets with standardization.
    dictionary_standard = { 'xs_int': xs_int_standard, 'ys_int': ys_int_standard,
              'Matrix_W': Matrix_W, 'num_N': num_N, 
              'mean': mean, 'std': std, 'minBB_1st': minBB_1st, 'minBB_2nd': minBB_2nd, 'minBB_mid2': minBB_mid2} 
                               

    file_standard = f'{folder_data}/initial_Data_{val_num_W}_{val_dim}_{val_standard}_{val_k}_{val_anneal}.bz2'
                                                                           
    with open(file_standard, 'wb') as fl_standard:
        pickle.dump(dictionary_standard, fl_standard)